In [1]:
import numpy as np
import tensorflow as tf

from tensorflow import keras

In [2]:
tf.config.list_physical_devices('GPU')

[]

In [3]:
import os

from typing import List

data_path = 'data/only_side_moves'
def load_data(type: str):
  dir_path: str = os.path.join(data_path, type) # ex: data/inputs
  data = {}
  for f in os.listdir(dir_path):
    data[type + '_' + f.replace('.npy', '')] = np.load(os.path.join(dir_path, f)).astype(np.float32)
    
  return data

inputs = load_data('input')
outputs = load_data('output')

move_dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

move_dataset.shuffle(len(move_dataset))

train_size: int = int(0.7 * len(move_dataset))
batch_size = 256
train_dataset = move_dataset.take(train_size).batch(batch_size)
test_dataset = move_dataset.skip(train_size).batch(batch_size)

# print(next(iter(move_dataset.take(1))))
# inputs, outputs = next(iter(move_dataset.take(1)))
# print(inputs)

def print_dataset(name, dataset):
  print('name:', name)
  print('length:', len(dataset))
  # print('dtype:', *[e.dtype for e in dataset[0]])
  print('sample:')
  inputs, outputs = next(iter(move_dataset.take(1)))
  print('inputs:')
  for k, v in inputs.items():
    print(k, v)
  print('outputs:')
  for k, v in outputs.items():
    print(k, v)
  print()

print_dataset('train_dataset', train_dataset)
# print_dataset('test_dataset', test_dataset)

name: train_dataset
length: 3
sample:
inputs:
input_dir tf.Tensor([0. 0.], shape=(2,), dtype=float32)
input_vel tf.Tensor([0. 0.], shape=(2,), dtype=float32)
input_wall tf.Tensor(
[[1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [4]:
from keras import Model
from keras.layers import Input, Dense, Flatten, Concatenate

input_dir = Input(2, name='input_dir')
input_vel = Input(2, name='input_vel')
input_wall = Input((18,24), name='input_wall')

x = Flatten()(input_wall)
x = Dense(40, activation='tanh', name='wall_features')(x)
x = Concatenate()([input_dir, input_vel, x])
x = Dense(40, activation='tanh', name='wall_feat_and_inputs')(x)
x = Concatenate()([input_dir, input_vel, x])
output_dpos = Dense(2, name='output_dpos')(x)
output_vel = Dense(2, name='output_vel')(x)

model = Model(inputs=[input_dir, input_vel, input_wall], outputs=[output_dpos, output_vel], name='move_model')

model.compile(loss=['mean_squared_error', 'mean_squared_error'], optimizer='sgd')
model.summary()

Model: "move_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_wall (InputLayer)        [(None, 18, 24)]     0           []                               
                                                                                                  
 flatten (Flatten)              (None, 432)          0           ['input_wall[0][0]']             
                                                                                                  
 input_dir (InputLayer)         [(None, 2)]          0           []                               
                                                                                                  
 input_vel (InputLayer)         [(None, 2)]          0           []                               
                                                                                         

In [6]:
history = model.fit(train_dataset, epochs=1000, validation_data=test_dataset)


Epoch 1/1000
3/3 [==============================] - 0s 16ms/step - loss: 0.0107 - output_dpos_loss: 0.0059 - output_vel_loss: 0.0048 - val_loss: 0.1843 - val_output_dpos_loss: 0.0182 - val_output_vel_loss: 0.1660
Epoch 2/1000
3/3 [==============================] - 0s 15ms/step - loss: 0.0107 - output_dpos_loss: 0.0059 - output_vel_loss: 0.0048 - val_loss: 0.1843 - val_output_dpos_loss: 0.0183 - val_output_vel_loss: 0.1660
Epoch 3/1000
3/3 [==============================] - 0s 16ms/step - loss: 0.0107 - output_dpos_loss: 0.0059 - output_vel_loss: 0.0048 - val_loss: 0.1843 - val_output_dpos_loss: 0.0183 - val_output_vel_loss: 0.1660
Epoch 4/1000
3/3 [==============================] - 0s 25ms/step - loss: 0.0107 - output_dpos_loss: 0.0059 - output_vel_loss: 0.0048 - val_loss: 0.1843 - val_output_dpos_loss: 0.0183 - val_output_vel_loss: 0.1660
Epoch 5/1000
3/3 [==============================] - 0s 69ms/step - loss: 0.0107 - output_dpos_loss: 0.0059 - output_vel_loss: 0.0048 - val_loss: 0.1

In [20]:
for inputs, outputs in test_dataset:
  # print(inputs['input_vel'])
  predictions = model.predict(inputs)
  
  print(predictions[0])
  for prd in predictions[0]:
    print(prd['output_dpos'])
    break
  break
  # print('Actual output:', outputs['output_dpos'])
  # print('Predicted output:', predictions[0])
  
  


8/8 [==============================] - 0s 3ms/step
[[-4.61010784e-02 -1.25583076e+00]
 [ 3.74192782e-02 -1.35749364e+00]
 [ 7.76686668e-02 -1.24593461e+00]
 [ 9.35073644e-02 -1.20877397e+00]
 [ 9.75228101e-02 -1.17260921e+00]
 [-1.04322687e-01 -1.32103693e+00]
 [-2.50609040e-01 -1.25020671e+00]
 [-1.87549636e-01 -1.38293493e+00]
 [-5.38682580e-01 -1.47380090e+00]
 [-4.92218018e-01 -1.33648705e+00]
 [-3.47929060e-01 -1.49724102e+00]
 [-3.48714232e-01 -1.38635933e+00]
 [-3.66915941e-01 -1.39300275e+00]
 [-4.10701096e-01 -1.51903188e+00]
 [-3.93964887e-01 -1.35258436e+00]
 [-4.22492117e-01 -1.40493119e+00]
 [-4.52632070e-01 -1.45384026e+00]
 [-4.73883897e-01 -1.46170926e+00]
 [-4.63349760e-01 -1.36035192e+00]
 [-5.13035417e-01 -1.46941125e+00]
 [-5.29717982e-01 -1.46083319e+00]
 [-5.15029252e-01 -1.36222708e+00]
 [-5.51972806e-01 -1.42092991e+00]
 [-5.78603446e-01 -1.44456351e+00]
 [-5.91322958e-01 -1.42966163e+00]
 [-6.02991760e-01 -1.41419578e+00]
 [-6.50730789e-01 -1.48547208e+00]
 [-6